In [ ]:
import darts
from darts import TimeSeries
from darts.models import ExponentialSmoothing
import pandas as pd
import numpy as np
import time
import httpx
import json
import sqlite3
import matplotlib as plt

In [2]:
# obtained from fuzzworks
invData = pd.read_csv('../invTypes.csv')

In [3]:
def getMarketPrices(inplace=False):
    print('Fetching market prices')
    r = httpx.get(
        'https://esi.evetech.net/latest/markets/prices/?datasource=tranquility',)
    if (r.status_code == 200):
        df = pd.read_json(r.text)
        df['date'] = dt.datetime.now()
        con = sqlite3.connect('../data/data.db')
        df.to_sql(con=con, name="market_data", if_exists="append")
        con.close()
        if (inplace):
            return (df)
        # cur = con.cursor()
        # cur.execute('select * from market_data')
        # results = cur.fetchall()
        # print(results)
    else:
        print(f"Request did not succeed! {r.status_code}")
        rjson = json.loads(r.text)
        if (rjson['timeout'] != None or 0):
            time.sleep((rjson['timeout']/1000))
# getMarketPrices()
# adjusted_price, average_price, type_id

In [4]:
def getItemPrices(id):
    # refresh database with new data
    getMarketPrices()
    con = sqlite3.connect('../data.db')
    # df = pd.read_sql_table(table_name='market_data',con=con,parse_dates='date')
    # cur = con.cursor()
    # cur.execute("select * from market_data")
    df = pd.read_sql('select * from market_data', con, parse_dates=['date'])
    selected_df = df.loc[df['type_id'] == id]
    if (len(selected_df) != 0):
        return (selected_df)
    else:
        print('Sorry, either this is not a valid item id or something went wrong. :P')

In [5]:
def getIDInfo(id):
    selected_row = invData.loc[invData['typeID'] == id]
    return selected_row
# getIDInfo(32772)

In [6]:
def getIdFromName(name):
    # NOTE: This function could be improved with a better search engine
    selected_rows = invData[invData['typeName'].str.contains(name, False)]
    selected_rows = selected_rows[['typeName', 'typeID']]
    return (selected_rows)
# getIdFromName('Booster')

In [7]:
def resample_one_minute(df):
    # Make sure the dataframe has a datetime index
    df.index = pd.to_datetime(df.index)
    # Resample the dataframe on one minute intervals
    df_resampled = df.resample('1T').mean()
    return df_resampled

ValueError: The time index of the provided DataArray is missing the freq attribute, and the frequency could not be directly inferred. This probably comes from inconsistent date frequencies with missing dates. If you know the actual frequency, try setting `fill_missing_dates=True, freq=actual_frequency`. If not, try setting `fill_missing_dates=True, freq=None` to see if a frequency can be inferred.


ValueError: The time index of the provided DataArray is missing the freq attribute, and the frequency could not be directly inferred. This probably comes from inconsistent date frequencies with missing dates. If you know the actual frequency, try setting `fill_missing_dates=True, freq=actual_frequency`. If not, try setting `fill_missing_dates=True, freq=None` to see if a frequency can be inferred.

In [ ]:
#Fix data for forecasting
con = sqlite3.connect('../data.db')
databaseframe = pd.read_sql(
    'select * from market_data', con, parse_dates=['date'])
con.close()
fixedData = resample_one_minute(databaseframe)

In [ ]:
# con = sqlite3.connect('../data.db')
# databaseframe = pd.read_sql('select * from market_data', con, parse_dates=['date'])
# con.close()
series = TimeSeries.from_dataframe(df=fixedData, time_col='date')
train, val = series.split_after(.6)
model_es = ExponentialSmoothing()
model_es.fit(train)
probabilistic_forecast = model_es.predict(len(val), num_samples=500)

series.plot(label="actual")
probabilistic_forecast.plot(label="probabilistic forecast")
plt.legend()
plt.show()
